<a href="https://colab.research.google.com/github/exkuretrol/concat-teams-mp4-and-generate-subtitles/blob/main/whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Onedrive 設定

下面是連結 Onedrive 的設定，如果是使用 Google Drive 就不需要執行它。

詳細設定可以參考：[Mount OneDrive to Google Colab. Hi readers, | by Pratik Mukherjee | Medium](https://medium.com/@pratikmukherjee32/mount-onedrive-to-google-colab-f941a3a96a79)

In [1]:
!wget https://downloads.rclone.org/v1.62.2/rclone-v1.62.2-linux-amd64.deb &> /dev/null
!apt install ./rclone-v1.62.2-linux-amd64.deb &> /dev/null
!sudo mkdir /content/onedrive
!sudo apt install fuse3 &> /dev/null

In [ ]:
!rclone config

In [3]:
!nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive &

nohup: appending output to 'nohup.out'


### 依賴套件

In [4]:
!pip install git+https://github.com/openai/whisper.git &> /dev/null

import os, re
import torch
from pathlib import Path

import whisper
from whisper.utils import get_writer

In [5]:
#@title 主要設定 { run: "auto" }

#@markdown 使用的容器，現有的設定有 `gdrive`，Google Drive；`onedrive`，Onedrive；`local`，colab 目前 session 的檔案
container = "onedrive" #@param ["gdrive", "onedrive","local"]

#@markdown 影片的檔案路徑 
filepath = "/content/onedrive/\u9298\u50B3\u5927\u5B78/\u8AB2\u7A0B\u9304\u5F71/\u7DB2\u7AD9\u4F3A\u670D\u5668\u6982\u8AD6/\u7DB2\u7AD9\u4F3A\u670D\u5668\u6982\u8AD6-W7.mp4" #@param {type:"string"}

#@markdown 影片的語言，中文 `zh` 英文 `en`，其他語言請參閱[whisper/tokenizer.py at main · openai/whisper](https://github.com/openai/whisper/blob/main/whisper/tokenizer.py)
lang = "zh" #@param {type:"string"}

#@markdown 是否要產生 .srt 字幕檔
srt = True #@param {type:"boolean"}

#@markdown 是否要產生 .vtt 字幕檔
vtt = False #@param {type:"boolean"}

#@markdown 是否要產生 .tsv 字幕檔
tsv = False #@param {type:"boolean"}

#@markdown 是否要下載產生的字幕檔
download = True #@param {type:"boolean"}

#@markdown 是否要顯示產生字幕的進度
verbose = False #@param {type:"boolean"}

#@markdown 使用的模型，多語系現有的模型有：`tiny`、`base`、`small`、`medium`與`large`；英文語系現有的模型有：`tiny.en`、`base.en`、`small.en`與`medium.en`。請參閱[Available models and languages](https://github.com/openai/whisper#available-models-and-languages)
model_type = "small" #@param ["tiny", "base", "small", "medium", "large", "tiny.en", "base.en", "small.en", "medium.en"]


In [12]:
# Use CUDA, if available
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load the desired model
model = whisper.load_model(model_type).to(DEVICE)

def transcribe_file(model, lang, file, srt, vtt, tsv, download, verbose):
  file_path = Path(file)
  print(f"Transcribing file: {file_path}\n")

  options = dict(
    highlight_words = False,
    max_line_width = None,
    max_line_count = None
  )

  output_directory = file_path.parent

  # Run Whisper
  result = model.transcribe(file, verbose = verbose, language = lang)

  if srt:
    print(f"\nCreating SRT file")
    srt_writer = get_writer("srt", output_directory)
    srt_writer(result, str(file_path.stem), options)

  if vtt:
    print(f"\nCreating VTT file")
    vtt_writer = get_writer("vtt", output_directory)
    vtt_writer(result, str(file_path.stem), options)

  if tsv:
    print(f"\nCreating TSV file")

    tsv_writer = get_writer("tsv", output_directory)
    tsv_writer(result, str(file_path.stem), options)

  if download:
    from google.colab import files
    
    stem = file_path.stem

    for colab_file in output_directory.glob(f"{stem}*"):
      if colab_file.suffix in [".srt", ".vtt", ".tsv"]:
        print(f"Downloading {colab_file}")
        files.download(str(colab_file))
    
  return result

再開始產生字幕之前請注意，如果是長時間的影片可能會產生到一半就中斷，這時候可以用 Javascript 設定每幾秒自動點擊一次網頁上的元素，藉此維持網頁 session。

操作說明如下：

1. 打開這個網頁的開發者工具

2. 切換到終端機分頁

3. 貼上以下的 Javascript

> 不過我有看到要手動按的你不是機器人🥲，所以這個方法不一定每次都管用

```javascript
function KeepClicking() {
  console.log("Clicking");
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}
setInterval(KeepClicking, 60000)
```

In [13]:
if container == "gdrive":
  from google.colab import drive
  drive.mount('/content/drive') 

elif container in ("onedrive", "local"):
  pass

result = transcribe_file(model, lang, filepath, srt, vtt, tsv, download, verbose)


Transcribing file: /content/onedrive/銘傳大學/課程錄影/網站伺服器概論/網站伺服器概論-W7.mp4



 98%|█████████▊| 780364/792364 [09:47<00:09, 1328.34frames/s]
